In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'

In [2]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [3]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [4]:
df.head(10)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь
3,"«Сачхере» — «Абердин» И тот, и другой клуб оде...",1.45,Айнтрахт - 3 и более гола,Лига Ставок,2019-07-25 17:00:00,Лига Европы УЕФА,Флора,Айнтрахт Франкфурт
4,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
5,Июль подходит к концу. Скоро мадридское дерби ...,1.62,Атлетико забьет 1 или 2 гола: да,Фонбет,2019-07-27 00:06:00,Международный Кубок чемпионов,Реал Мадрид,Атлетико Мадрид
6,"«Ахмат», который доставил много проблем команд...",1.71,Ахмат забьет 1 или 2 гола: да,Пари-Матч,2019-07-14 18:30:00,Премьер-Лига,Ахмат,Краснодар
7,"Никто толком не представляет, в каких составах...",2.45,Бавария забьет в обоих таймах,Лига Ставок,2019-07-18 03:00:00,Международный Кубок чемпионов,Арсенал,Бавария
8,"Матч вроде бы товарищеский, но - это и репутац...",3.25,Барселона забьет в каждом тайме,Лига Ставок,2019-07-23 10:30:00,Клубные товарищеские матчи,Барселона,Челси
9,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония


In [5]:
unique_leagues = df['league_name_in_russian'].unique().tolist()
len(unique_leagues)

290

In [6]:
unique_teams1 = df['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

3803

In [7]:
unique_teams2 = df['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

3760

In [8]:
unique_teams = unique_teams1 + unique_teams2
len(set(unique_teams))

4454

### Команда 1 и 2 (список + прямой мэтч)

In [20]:
teams1_match = np.zeros(df.shape[0])
teams2_match = np.zeros(df.shape[0])

https://stackoverflow.com/questions/15852436/how-to-find-the-2nd-max-of-a-counter-python

In [22]:
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            #print(token)
            if token in unique_teams:
                count_tokens.append(token)
        #print(count_tokens)
        c = Counter(count_tokens)
        #if count_tokens == 2
        #print(df['team_of_season_1_name_in_russian'][i], df['team_of_season_2_name_in_russian'][i])
        #print(c)
        #print(max(c))
        #print(c.most_common()[0:2])
        #print(text)
        try:
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
        except:
            continue
        try:
            max_w2 = sorted(c, key=c.get)[-2]
            #print(max_w2)
        except:
            continue
        
        #print('мэтч команды 1')
        if df['team_of_season_1_name_in_russian'][i] == max_w1 or df['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1
            #print(df['team_of_season_1_name_in_russian'][i], max_w1)
            #print(df['team_of_season_1_name_in_russian'][i], max_w2)
        
        #print('мэтч команды 2')
        if df['team_of_season_2_name_in_russian'][i] == max_w1 or df['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1
            #print(df['team_of_season_2_name_in_russian'][i], max_w1)
            #print(df['team_of_season_2_name_in_russian'][i], max_w2)
        
        

702908it [5:24:59, 36.05it/s] 


In [25]:
df['list_direct_match_for_teams1'] = teams1_match

In [26]:
df['list_direct_match_for_teams2'] = teams2_match

In [27]:
success1 = df[df['list_direct_match_for_teams1'] == 1].shape[0]
success2 = df[df['list_direct_match_for_teams2'] == 1].shape[0]

In [29]:
all_vals = df['list_direct_match_for_teams1'].shape[0]

In [30]:
result_team1 = success1 / all_vals
print(result_team1)

0.20481058687623416


In [31]:
result_team2 = success2 / all_vals
print(result_team2)

0.2051363763109824


### Лиги

In [32]:
leagues_match = np.zeros(df.shape[0])

In [44]:
%%time 
for i, text in tqdm(enumerate(df['result.tsv'][:1000].values)):
    n_words_league = len(word_tokenize(df['league_name_in_russian'][i]))
    N = n_words_league
    words = word_tokenize(text)
    words = [word for word in words if word not in punct ]
    grams = [words[i:i+N] for i in range(len(words)-N+1)]

    for gram in grams: 
        count_tokens = []
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if joined_gram in unique_leagues:
            count_tokens.append(token)
    
    if count_tokens:
        print(count_tokens)
    
            
#         c = Counter(count_tokens)
#         print(c)

1000it [00:24, 40.66it/s]


Wall time: 24.6 s
